This notebook shows the code for generating the baseline for FHMM and LBM

In [1]:
from hmmlearn import hmm

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 20 days


In [2]:
from create_df import read_df
df, dfc, all_homes, appliance_min, national_average = read_df()

In [3]:
import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings

In [4]:
warnings.filterwarnings("ignore")

In [5]:
from nilmtk import *
import os
import nilmtk

In [7]:
st = pd.HDFStore("/Users/nipunbatra/Downloads/wiki-temp.h5")

In [10]:
models = {}

for appliance in ["air conditioner", "fridge", "spin dryer", 
                  "dish washer", "washing machine", "light"]:
    #for appliance in [ "dish washer"]:
    print appliance, "training"
    o = []
    for building_num in range(1, len(ds.buildings)):
        
        building = ds.buildings[building_num]
        elec = building.elec
        try:
            dft = elec.select_using_appliances(type=appliance).load(cols=[('power','active')], sample_period=900).next().squeeze()
            appl_power = dft.dropna().values.reshape(-1,1)
            activation = (dft>10).sum()*1.0/len(df)
            if appliance in ["washer dryer", "dishwasher"]:
                o.append(appl_power)
                
            else:
                if activation>0.05:
                    o.append(appl_power)
        except:
            pass
    
    if len(o)>1:
        o = np.array(o)
        mod = hmm.GaussianHMM(3, "full")
        mod.fit(o)
        models[appliance] = mod
        print "Means for %s are" %appliance
        print mod.means_
    else:
        print "Not enough samples for %s" %appliance

air conditioner training
Loading data for meter ElecMeterID(instance=9, building=1, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=21, building=2, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=15, building=3, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=20, building=4, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=14, building=5, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=19, building=6, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=16, building=7, dataset='Dataport')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=11, building=8, datase

In [8]:
from nilmtk.disaggregate.fhmm_exact import sort_learnt_parameters

In [9]:
new_learnt_models = OrderedDict()
for appliance, appliance_model in models.iteritems():
    startprob, means, covars, transmat = sort_learnt_parameters(
                    appliance_model.startprob_, appliance_model.means_,
                    appliance_model.covars_, appliance_model.transmat_)
    new_learnt_models[appliance] = hmm.GaussianHMM(
                startprob.size, "full", startprob, transmat)
    new_learnt_models[appliance].means_ = means
    new_learnt_models[appliance].covars_ = covars

NameError: name 'models' is not defined

In [ ]:
from nilmtk.disaggregate.fhmm_exact import create_combined_hmm

In [ ]:
learnt_model_combined = create_combined_hmm(new_learnt_models)

In [ ]:
from nilmtk.disaggregate.fhmm_exact import FHMM

In [16]:
f = FHMM()

In [17]:
f.model = learnt_model_combined
f.individual = new_learnt_models

In [18]:
import pickle

In [19]:
pickle.dump(f, open( "../fhmm_model_all.p", "wb" ))

In [8]:
all_homes_all_appliance = []
for appliance in ["fridge","hvac","dr","dw","wm","light"]:
    all_homes_all_appliance = np.union1d(all_homes_all_appliance, all_homes[appliance])

In [21]:
import sys

In [22]:
st_out = pd.HDFStore("../fhmm_out.h5")
done = -1
for home in all_homes_all_appliance[:]:
    done = done +1
    print "%0.2f" %(100*done/len(all_homes_all_appliance))
    sys.stdout.flush()
    try:
        mains_df = st['/%d' %home][['use']]['2013'].resample("15T")
        pred = f.disaggregate_chunk(mains_df).resample("1M", how="sum").mul(0.000017).mul(15)
        pred.index = range(1, 13)
        pred = pred.rename(columns={'air conditioner':'hvac','dish washer':'dw',
                                    'spin dryer':'dr','washing machine':'wm'})
        st_out['%d' %home] = pred
    except:
        pass

0.00
2.00
4.00
6.00
9.00
11.00
13.00
16.00
18.00
20.00
23.00
25.00
27.00
30.00
32.00
34.00
37.00
39.00
41.00
44.00
46.00
48.00
51.00
53.00
55.00
58.00
60.00
62.00
65.00
67.00
69.00
72.00
74.00
76.00
79.00
81.00
83.00
86.00
88.00
90.00
93.00
95.00
97.00


In [9]:
st_out = pd.HDFStore("../fhmm_out.h5")
fhmm_accur = {}
for appliance in ["hvac","fridge","dw","wm","light","dr"]:
    print appliance
    
    if appliance =="hvac":
        start, end = 5, 11
    else:
        start, end = 1, 13
    out_ap = {}
    accuracies = {}
    for home in all_homes[appliance][:]:
        out_ap[home] = {}
        pred = st_out['/%d' %home][appliance]
        for month in range(start, end):
            out_ap[home][month] = {}
            gt = df.ix[home]['%s_%d' %(appliance, month)]
            out_ap[home][month]['gt'] = gt
            out_ap[home][month]['pred'] = pred.ix[month]
        a = pd.DataFrame(out_ap[home]).T
        e = (a['gt']-a['pred']).abs().div(a['gt'])*100
        accur = 100-e
        accur[accur<0] = 0
        accuracies[home] = accur
    fhmm_accur[appliance] = pd.DataFrame(accuracies)

hvac
fridge
dw
wm
light
dr


In [10]:
accuracy_store = pd.HDFStore("accuracy.h5")

In [11]:
accuracy_store

<class 'pandas.io.pytables.HDFStore'>
File path: accuracy.h5
/N-NILM            wide_table   (typ->appendable,nrows->420,ncols->6,indexers->[major_axis,minor_axis])

In [12]:
accuracy_store.append("FHMM",pd.Panel(fhmm_accur))

In [13]:
accuracy_store

<class 'pandas.io.pytables.HDFStore'>
File path: accuracy.h5
/FHMM              wide_table   (typ->appendable,nrows->510,ncols->6,indexers->[major_axis,minor_axis])
/N-NILM            wide_table   (typ->appendable,nrows->420,ncols->6,indexers->[major_axis,minor_axis])

In [18]:
st_out = pd.HDFStore("../fhmm_lbm.h5")

for home in all_homes_all_appliance[:]:
    print home
    try:
        pred = pd.read_csv("/Users/nipunbatra/Dropbox/Public/notebooks/misc/%d.csv" %home, index_col=0)
        pred.index = pd.to_datetime(pred.index)
        pred = pred['2013']
        pred = pred.resample("1M", how="sum").mul(0.000017).mul(15)
        pred.index = range(1, 13)
        pred = pred[['hvac','dw','dr','fridge','wm','light']]
        
        st_out['%d' %home] = pred
    except Exception, e:
        print e

26.0
94.0
370.0
410.0
545.0
624.0
1283.0
1334.0
1642.0
1953.0
2129.0
2156.0
2242.0
2470.0
2814.0
3367.0
3456.0
3482.0
3723.0
3967.0
4732.0
4767.0
5357.0
5371.0
5746.0
5785.0
5814.0
6072.0
6636.0
6836.0
6910.0
7016.0
7731.0
7769.0
7866.0
8317.0
9052.0
9609.0
9654.0
9922.0
9933.0
9934.0
9982.0


In [14]:
st_out = pd.HDFStore("../fhmm_lbm.h5")
lbm_accur = {}
for appliance in ["hvac","fridge","dw","wm","light","dr"]:
    print appliance
    
    if appliance =="hvac":
        start, end = 5, 11
    else:
        start, end = 1, 13
    out_ap = {}
    accuracies = {}
    for home in all_homes[appliance][:]:
        out_ap[home] = {}
        pred = st_out['/%d' %home][appliance]
        for month in range(start, end):
            out_ap[home][month] = {}
            gt = df.ix[home]['%s_%d' %(appliance, month)]
            out_ap[home][month]['gt'] = gt
            out_ap[home][month]['pred'] = pred.ix[month]
        a = pd.DataFrame(out_ap[home]).T
        e = (a['gt']-a['pred']).abs().div(a['gt'])*100
        accur = 100-e
        accur[accur<0] = 0
        accuracies[home] = accur
    lbm_accur[appliance] = pd.DataFrame(accuracies)

hvac
fridge
dw
wm
light
dr


In [15]:
accuracy_store.append("LBM",pd.Panel(lbm_accur))

In [16]:
accuracy_store

<class 'pandas.io.pytables.HDFStore'>
File path: accuracy.h5
/FHMM              wide_table   (typ->appendable,nrows->510,ncols->6,indexers->[major_axis,minor_axis])
/LBM               wide_table   (typ->appendable,nrows->510,ncols->6,indexers->[major_axis,minor_axis])
/N-NILM            wide_table   (typ->appendable,nrows->420,ncols->6,indexers->[major_axis,minor_axis])

In [17]:
accuracy_store.close()